In [1]:
import pandas as pd
import numpy as np
import sys
from urllib.request import urlopen
from datetime import date, timedelta
import json
import io
import mysql.connector

## Reading data from API

In [2]:
# #ליאור הרצה עם הגישה שלו, לא למחוק!!
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   password="CoronaProject1234!",
#   database="corona"
# )

# print(mydb)

In [3]:
# קישור ל-MYSQL
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Corona1234!",
  database="corona"
)

In [4]:
# בדיקה האם יש כבר טבלה כזאת במסד
mycursor = mydb.cursor()

mycursor.execute("SHOW TABLES")

tables_list = []
for table in mycursor:
      tables_list+=table
tables_list

['cities_names_codes', 'city_status_vaccine_total']

In [5]:
flag_table_in_DB = False

if 'city_status_vaccine_total' in tables_list:
    flag_table_in_DB = True
    print ('Table already exist')
else:
    mycursor = mydb.cursor()
    mycursor.execute("CREATE TABLE city_status_vaccine_total (City_Code INT NOT NULL, Date Date NOT NULL, Accumulated_Verified_Cases INT, Accumulated_Recovered INT,Accumulated_Deaths INT, Accumulated_Number_Of_Tests INT, Accumulated_Vaccination_First_Dose INT, Accumulated_Vaccination_Second_Dose INT, Accumulated_Vaccination_Third_Dose INT, City_Color TEXT, Final_Score DOUBLE,   PRIMARY KEY (City_Code, Date))")
    print ('Table created')

Table already exist


In [6]:
# שליפת היום האחרון שמכיל נתונים ב-SQL
mycursor = mydb.cursor()
mycursor.execute("SELECT max(Date) FROM city_status_vaccine_total")    
myresult = mycursor.fetchall()
if len(str(myresult)) > 1:
    last_date_db = myresult
    last_date_db = last_date_db[0][0]
else:
    last_date_db= date(2020,3, 10) # 11/03/2020 FIRST DAY OF COVID-19 DATA
print (last_date_db)

2022-08-14


In [7]:
# פונקציה שמקבלת תאריך ומחשבת מה היום הבא, 
def calculate_next_day(last_day):
    next_day = date(last_day.year,last_day.month,last_day.day) + timedelta(days=1)
    return next_day

In [8]:
# הורדת מידע רמזור = סטטוס
df_Corona_City_Status = pd.DataFrame()
next_day = calculate_next_day(last_date_db)
while (1==1):
    resource_id_vaccine = '8a21d39d-91e3-40db-aca1-f73f7ab1df69'
    url = 'https://data.gov.il/api/3/action/datastore_search?resource_id={}&q={}&limit=5000'.format(resource_id_vaccine,next_day)
    fileobj = urlopen(url)
    # convert to utf-8 and to dict
    with urlopen(url) as r, \
         io.TextIOWrapper(r, encoding=r.headers.get_content_charset('utf-8')) as file:
        ramzor_result = json.load(file)
        json_ramzor_data = ramzor_result['result']['records']
    if (len(json_ramzor_data) < 1):
        x = 0
        break
    df_ramzor = pd.DataFrame.from_dict(json_ramzor_data)
    df_Corona_City_Status = df_Corona_City_Status.append(df_ramzor,ignore_index=True)
    print(next_day)
    next_day = calculate_next_day(next_day)


2022-08-15
2022-08-16
2022-08-17
2022-08-18
2022-08-19
2022-08-20
2022-08-21
2022-08-22
2022-08-23


In [9]:
df_Corona_City_Status = df_Corona_City_Status.drop_duplicates(subset=['Date', 'City_Code'], keep='last')

In [10]:
# הוצאת התאריך האחרון של טבלת הרמזור שהורדה
# על מנת להוריד נתונים עד לאותו תאריך מטבלת חיסונים
last_date_ramzor = max(df_Corona_City_Status['Date'])
last_date_ramzor = pd.to_datetime(last_date_ramzor).date()
last_date_ramzor


datetime.date(2022, 8, 23)

In [11]:
# מוריד מידע חיסונים
df_vaccine = pd.DataFrame()
next_day = calculate_next_day(last_date_db)

while (1==1):
    resource_id_vaccine = 'd07c0771-01a8-43b2-96cc-c6154e7fa9bd'
    url = 'https://data.gov.il/api/3/action/datastore_search?resource_id={}&q={}&limit=5000'.format(resource_id_vaccine,next_day)
    fileobj = urlopen(url)
    # convert to utf-8 and to dict
    with urlopen(url) as r, \
         io.TextIOWrapper(r, encoding=r.headers.get_content_charset('utf-8')) as file:
        vaccine_result = json.load(file)
        json_vaccine_data = vaccine_result['result']['records']
    if ((next_day >= last_date_ramzor) or (len(json_vaccine_data) < 1)):
        x = 0
        break
    df_vaccine_tmp = pd.DataFrame.from_dict(json_vaccine_data)
    df_vaccine = df_vaccine.append(df_vaccine_tmp,ignore_index=True)
    print(next_day)
    next_day = calculate_next_day(next_day)
    

2022-08-15
2022-08-16
2022-08-17
2022-08-18
2022-08-19
2022-08-20
2022-08-21


In [12]:
# אם טבלת החיסונים ריקה = אין מידע - מפסיק את ריצת הקוד
if(len(df_vaccine)<1):
    print('There is no new data to update. Closing the script.')
    sys.exit()


In [13]:
## ניקוי המידע
# מוחק עמודות בינאריות, מס' מאושפזים ושם עיר 
df_vaccine.drop(columns=['new_cases_on_date','new_recoveries_on_date','new_hospitalized_on_date','new_deaths_on_date','new_diagnostic_tests_on_date','new_vacc_first_dose_on_date','new_vacc_second_dose_on_date','new_vacc_third_dose_on_date','accumulated_hospitalized','town','rank'], inplace=True)


In [14]:
# מחליף שמות של עמודות לפי ההגדרה באקסל שלנו
df_vaccine = df_vaccine.rename(columns={"town_code": "City_Code",
                           "agas_code": "Agas_Code",
                           "town": "City_name",
                           "date":"Date",
                           "accumulated_cases":"Accumulated_Verified_Cases",
                           "accumulated_recoveries":"Accumulated_Recovered",
                           "accumulated_deaths":"Accumulated_Deaths",
                           "accumulated_diagnostic_tests":"Accumulated_Number_Of_Tests",
                           "accumulated_vaccination_first_dose":"Accumulated_Vaccination_First_Dose",
                           "accumulated_vaccination_second_dose":"Accumulated_Vaccination_Second_Dose",
                           "accumulated_vaccination_third_dose":"Accumulated_Vaccination_Third_Dose"
                          })

In [15]:
# ממיר ערכים של 15> ל-7
df_vaccine.loc[df_vaccine['Accumulated_Verified_Cases'] == '<15', 'Accumulated_Verified_Cases'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Recovered'] == '<15', 'Accumulated_Recovered'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Deaths'] == '<15', 'Accumulated_Deaths'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Number_Of_Tests'] == '<15', 'Accumulated_Number_Of_Tests'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Vaccination_First_Dose'] == '<15', 'Accumulated_Vaccination_First_Dose'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Vaccination_Second_Dose'] == '<15', 'Accumulated_Vaccination_Second_Dose'] = 7
df_vaccine.loc[df_vaccine['Accumulated_Vaccination_Third_Dose'] == '<15', 'Accumulated_Vaccination_Third_Dose'] = 7

In [16]:
# המרת סוגי הנתונים בשדות
df_vaccine['Date'] = pd.to_datetime(df_vaccine['Date'])

cols = ['Accumulated_Verified_Cases','Accumulated_Recovered','Accumulated_Deaths','Accumulated_Number_Of_Tests','Accumulated_Vaccination_First_Dose','Accumulated_Vaccination_Second_Dose','Accumulated_Vaccination_Third_Dose']
df_vaccine[cols] = df_vaccine[cols].apply(pd.to_numeric)
df_vaccine[cols] = df_vaccine[cols].astype(int)


In [17]:
# הופך נתונים שהם null ל-0
df_vaccine.fillna(0)

,_id,City_Code,Agas_Code,Date,Accumulated_Verified_Cases,Accumulated_Recovered,Accumulated_Deaths,Accumulated_Number_Of_Tests,Accumulated_Vaccination_First_Dose,Accumulated_Vaccination_Second_Dose,Accumulated_Vaccination_Third_Dose
0,1283154,8400,0214,2022-08-15,2034,2022,7,23621,2890,2723,2162
1,1447650,9500,0003,2022-08-15,3099,3086,7,33634,4102,3908,3069
2,953178,6600,0415,2022-08-15,1997,1992,7,21072,2613,2468,1848
3,968376,6600,0615,2022-08-15,1524,1509,7,14481,1948,1803,1337
4,506892,3000,1016,2022-08-15,1762,1755,7,16496,1802,1464,990
...,...,...,...,...,...,...,...,...,...,...,...
11609,846708,6200,0212,2022-08-21,1695,1672,15,19639,2751,2619,2083
11610,92082,326,0,2022-08-21,1250,1240,7,16660,1893,1816,1508
11611,880770,6300,0031,2022-08-21,1728,1724,7,21766,2642,2578,2103
11612,1114104,7400,0413,2022-08-21,2149,2139,7,26134,3384,3239,2610


In [18]:
# מקבץ לפי קוד עיר ותאריך וסוכם את כל הנתונים  
# (מונע מקוד עיר ותאריך להפוך לאינדקס)
df_vaccine_group = df_vaccine.groupby(['City_Code', 'Date'],as_index=False).sum()

In [19]:
# מוחק עמודה של קוד אזור  
try:
    df_vaccine_group.drop(['Agas_Code'],axis=1, inplace=True)
    df_vaccine_group.drop(columns=['_id'], inplace=True)
except:
    pass

In [20]:
dup = df_vaccine_group[df_vaccine_group.duplicated(['City_Code','Date'])]

In [21]:
df_vaccine_group = df_vaccine_group.drop_duplicates(subset=['Date', 'City_Code'], keep='last')

In [22]:
#  פונקציה שמוחקת מהטבלה את הערים שלא נמצאות בטבלה השניה

def delete_cities(df,cities_list):
    for i in cities_list:
        df = df.loc[df['City_Code']!=i]
    return df

In [23]:
cities_codes_to_drop_1 = ['190']
df_vaccine_group = delete_cities(df_vaccine_group,cities_codes_to_drop_1)

In [24]:
len(df_vaccine_group)

1828

# Clean the table and Combine with Ramzor table

In [25]:
df_Corona_City_Status.drop(columns=['_id','rank'], inplace=True)

In [26]:
# מוחק עמודות של שם עיר, ועמודה כפולה של בדיקות
df_Corona_City_Status.drop(columns=['Cumulated_number_of_diagnostic_tests','City_Name'], inplace=True)

In [27]:
# מחליף שמות של עמודות לפי ההגדרה במסד שלנו
df_Corona_City_Status = df_Corona_City_Status.rename(columns={
                           "Cumulative_verified_cases":"Accumulated_Verified_Cases",
                           "Cumulated_recovered":"Accumulated_Recovered",
                           "Cumulated_deaths":"Accumulated_Deaths",
                           "Cumulated_number_of_tests":"Accumulated_Number_Of_Tests",
                           "colour":"City_Color",
                           "final_score":"Final_Score",
                          })

In [28]:
# ממיר ערכים של 15> ל-7
df_Corona_City_Status.loc[df_Corona_City_Status['Accumulated_Verified_Cases'] == '<15', 'Accumulated_Verified_Cases'] = 7
df_Corona_City_Status.loc[df_Corona_City_Status['Accumulated_Recovered'] == '<15', 'Accumulated_Recovered'] = 7
df_Corona_City_Status.loc[df_Corona_City_Status['Accumulated_Deaths'] == '<15', 'Accumulated_Deaths'] = 7
df_Corona_City_Status.loc[df_Corona_City_Status['Accumulated_Number_Of_Tests'] == '<15', 'Accumulated_Number_Of_Tests'] = 7

In [29]:
# המרת סוגי הנתונים בשדות
df_Corona_City_Status['Date'] = pd.to_datetime(df_Corona_City_Status['Date'])
df_Corona_City_Status = df_Corona_City_Status.astype({"Accumulated_Verified_Cases": int})
df_Corona_City_Status = df_Corona_City_Status.astype({"Accumulated_Verified_Cases": int})
df_Corona_City_Status = df_Corona_City_Status.astype({"Accumulated_Recovered": int})
df_Corona_City_Status = df_Corona_City_Status.astype({"Accumulated_Deaths": int})
df_Corona_City_Status = df_Corona_City_Status.astype({"Accumulated_Number_Of_Tests": int})
df_Corona_City_Status = df_Corona_City_Status.astype({"City_Color": "string"})

In [30]:
# הופך נאל ללא רלוונטי
df_Corona_City_Status['City_Color'] = df_Corona_City_Status['City_Color'].fillna('')
df_Corona_City_Status['City_Color'] = df_Corona_City_Status['City_Color'].replace('NULL','Irrelevant')
df_Corona_City_Status['Final_Score'] = df_Corona_City_Status['Final_Score'].fillna(-1)
df_Corona_City_Status['Final_Score'] = df_Corona_City_Status['Final_Score'].replace('NULL',-1)

df_Corona_City_Status = df_Corona_City_Status.astype({"Final_Score": float})

In [31]:
# מוחק מהטבלה את הערים שלא נמצאות בטבלה השניה
cities_codes_to_drop_2 = ['190','649','694','939','961','963','964','965','966','967',
                          '968','997','1169','1170','1239','1375','3659','3768','3787',
                          '3793','9999','7000', '709', '7100', '7200', '7300', '7400', 
                          '7500', '7600', '763', '7700', '7800', '7900', '8000', '812',
                          '8200', '8300','831', '8400', '8500', '8600', '8700', '874']
df_Corona_City_Status = delete_cities(df_Corona_City_Status,cities_codes_to_drop_2)

In [32]:
#אורך טבלת חיסונים
len(df_vaccine_group)

1828

In [33]:
#אורך טבלת רמזור
len(df_Corona_City_Status)

2162

In [34]:
#איחוד של שתי הטבלאות, על סמך קוד עיר ותאריך
#עם פרמטר INNER
#ששומר שורות שמכילות ערכים רק בשתי הטבלאות  
merged_df = pd.merge(df_vaccine_group, df_Corona_City_Status, how='inner', on=['Date','City_Code'])
len(merged_df)

1674

In [35]:
new_dates = pd.unique(merged_df['Date'])
new_dates = pd.to_datetime(new_dates)

In [36]:
have_new_dates = False
for i in new_dates:
    if i > last_date_db:
        have_new_dates = True
if have_new_dates == False:
    print('There is no new data to update. Closing the script.')
    sys.exit()

In [37]:
dup = merged_df[merged_df.duplicated(['City_Code','Date'])]

In [38]:
merged_df = merged_df.drop_duplicates(subset=['Date', 'City_Code'], keep='last')

In [39]:
#מחיקה של עמודות שמכילות אותם נתונים 
#היו קצת שוני בין העמודות, מחקנו את הקטן יותר שהיה בטבלת החיסונים
merged_df.drop(['Accumulated_Verified_Cases_x','Accumulated_Recovered_x','Accumulated_Deaths_x','Accumulated_Number_Of_Tests_x'],axis=1, inplace=True)

In [40]:
#שינוי סדר העמודות, כדי שמספר נדבקים, מתים, מחלימים, בדיקות יהיה בתחילת הטבלה 
colomn_new_list = [
 'City_Code',
 'Date',
 'Accumulated_Verified_Cases_y',
 'Accumulated_Recovered_y',
 'Accumulated_Deaths_y',
 'Accumulated_Number_Of_Tests_y',
 'Accumulated_Vaccination_First_Dose',
 'Accumulated_Vaccination_Second_Dose',
 'Accumulated_Vaccination_Third_Dose',
 'City_Color',
 'Final_Score']
merged_df = merged_df[colomn_new_list]

In [41]:
# מחליף שמות של עמודות לפי ההגדרה באקסל שלנו
merged_df = merged_df.rename(columns={
                           "Accumulated_Verified_Cases_y": "Accumulated_Verified_Cases",
                           "Accumulated_Recovered_y":"Accumulated_Recovered",
                           "Accumulated_Deaths_y":"Accumulated_Deaths",
                           "Accumulated_Number_Of_Tests_y":"Accumulated_Number_Of_Tests",
                          })

In [42]:
merged_df['City_Color'] = merged_df['City_Color'].replace('ירוק','Green')
merged_df['City_Color'] = merged_df['City_Color'].replace('צהוב','Yellow')
merged_df['City_Color'] = merged_df['City_Color'].replace('כתום','Orange')
merged_df['City_Color'] = merged_df['City_Color'].replace('אדום','Red')

In [43]:
pd.unique(merged_df['City_Color'])

<StringArray>
['Orange', 'Yellow', 'Green', 'Red']
Length: 4, dtype: string

In [44]:
last_date_db = pd.to_datetime(last_date_db)
q = merged_df.Date < last_date_db 
print ('Number of rows that date is earlier than last_date_db: ',len(q[q == True])) 

Number of rows that date is earlier than last_date_db:  1


In [45]:
merged_df = merged_df.drop(merged_df[merged_df.Date < last_date_db].index)

In [46]:
merged_df.dtypes

City_Code                                      object
Date                                   datetime64[ns]
Accumulated_Verified_Cases                      int32
Accumulated_Recovered                           int32
Accumulated_Deaths                              int32
Accumulated_Number_Of_Tests                     int32
Accumulated_Vaccination_First_Dose              int32
Accumulated_Vaccination_Second_Dose             int32
Accumulated_Vaccination_Third_Dose              int32
City_Color                                     string
Final_Score                                   float64
dtype: object

In [47]:
# מכניס ל MYSQL

mycursor = mydb.cursor()

cols = "`,`".join([str(i) for i in merged_df.columns.tolist()])
number_rows_inserted = 0
# לולאה שמכניסה כל שורה בדאטאפריים לתוך המסד
for i,row in merged_df.iterrows():
    sql = "INSERT INTO `city_status_vaccine_total` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    mycursor.execute(sql, tuple(row))
    number_rows_inserted += 1
mydb.commit()

print ("number_rows_inserted: ",number_rows_inserted)

number_rows_inserted:  1673
